In [5]:
########################## Información general ##########################
###Eliminación por Bloques a través de métodos de Jacobi y GaussSiedel###
##############1.- Repositorio de Funciones y Librerías###################

# Instalación de librerías y paqueterías auxiliares
if(!require(pracma)){
    install.packages('pracma', repos = "http://cran.us.r-project.org")
}
if(!require(matrixcalc)){
    install.packages("matrixcalc")
}
library('matrixcalc')
library('pracma')

#################### Declaración de funciones generales##################

funcEsVectorValido <- function(mtrx, vct){
  # Valida si el vector (ya se de resultados o aproximaciones) cuenta
  # con la misma cantidad de filas que la matriz a evaluar.
  #
  # Parámetros
  # ----------
  # mtrx : matriz
  #    La matriz a evaluar
  # vct : vector
  #    El vector cuyo # de filas se comparará contra las de la matriz a evaluar
  #
  # Regresa
  # -------
  # boolean
  #    Una bandera a manera de booleano indicando si la cantidad de filas del
  #    vector es igual o no a la cantidad de filas de la matriz a evaluar.
  #

  # El vector b debe tener la misma cantidad de filas que la matriz
  bool_VectorValido = FALSE
  if (nrow(mtrx) == length(vct)){
    bool_VectorValido = TRUE
  }

  bool_VectorValido

}

# Valida la matriz sea cuadrada (nxn)
funcEsMatrizCuadrada <- function(mtrx){
  #
  # Parámetros
  # ----------
  # mtrx : matriz
  #    La matriz a evaluar
  #
  # Regresa
  # -------
  # boolean
  #    Una bandera a manera de booleano indicando si la cantidad de filas y
  #    columnas de la matriz son iguales.
  #

  # Si el número de filas es igual al número de columnas, es una matriz cuadrada
  if (nrow(mtrx)==ncol(mtrx)){
    bool_Valida <- TRUE
  }
  else{
    bool_Valida <- FALSE
  }

  bool_Valida

}

 # Busca algún cero en la diagonal principal
funcHayCeroEnDiagonal <- function(mtrx){
  # Valida si se presenta alún cero en la diagonal de la matriz
  #
  # Parámetros
  # ----------
  # mtrx : matriz
  #    La matriz a evaluar
  #
  # Regresa
  # -------
  # boolean
  #    Una bandera a manera de booleano indicando si se encontró un cero en
  #    la diagonal de la matriz.
  #

  bool_HayCero <- FALSE
  nbr_Filas <- nrow(mtrx)
  nbr_Cols <- ncol(mtrx)

  for (i in 1:nbr_Filas){
    for (j in 1:nbr_Cols){
      if ((i==j) && (mtrx[i,j]==0)){
        bool_HayCero <- TRUE
      }
    }
  }

  bool_HayCero

}

# Función que obtiene cada componente del vector
funcObtenerComponente <- function(i, n, mtrx_A, vct_X, vct_B){
  # Obtiene 1 solo componente del vector de aproximaciones
  #
  # Parámetros
  # ----------
  # i : número
  #    Índice del componente (del vector de aproximaciones) que se quiere obtener
  # n : número
  #    Dimensión de filas o columnas de la matriz
  # mtrx_A: matriz
  #    La matriz que se está evaluando
  # vct_X : vector
  #    Vector de aproximaciones
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  #
  # Regresa
  # -------
  # número
  #    El valor del componente indicado para el vector de aproximaciones
  #

  # Variable en la cual acumularemos el resutlado de la sumatoria
  nbr_Sumatoria = 0
  nbr_Final = 0

  # Variables con los términos agrupados de los elementos de la fórmula
  nbr_Termino1 = 0
  nbr_Termino2 = 0

  # Sumatoria de j a n para toda j != i
  for (j in 1:n){
    if (j != i ){

      # Operación de la sumatoria
      nbr_Termino1 = (-(mtrx_A[i,j] * vct_X[j]) / (mtrx_A[i,i]))

      # Acumulamos los valores
      nbr_Sumatoria = nbr_Sumatoria + nbr_Termino1

    }
  }

  # Terminada la sumatoria, se prepara un término extra
  nbr_Termino2 = (vct_B[i] / mtrx_A[i,i])

  # El resultado final es lo acumulado de la sumatoria más el otro término
  nbr_Final = nbr_Sumatoria + nbr_Termino2

  # Regresamos el resultado
  nbr_Final

}

funcInterCambiarFilasVct <- function(vctOrigen, nbr_FilaOrigen, nbr_FilaDestino){
  # Intercambia las filas de un vector
  #
  # Parámetros
  # ----------
  # mtrx_A : vector
  #    El vector donde se realizará el intercambio de filas
  # nbr_FilaOrigen : número
  #    Índice de la fila origen que se moverá a la fila destino
  # nbr_FilaDestino : número
  #    Índice de la fila destino que se moverá a la fila origen
  #
  # Regresa
  # -------
  # vector
  #    El vector con los valores intercambiados
  #

  # Se guarda el valor destino
  nbr_ValorTmp <- vctOrigen[nbr_FilaDestino]

  # Se pone el valor origen hacia el valor destino
  vctOrigen[nbr_FilaDestino] <- vctOrigen[nbr_FilaOrigen]

  # Se recupera el valor destino original y se pone en valor origen
  vctOrigen[nbr_FilaOrigen] <- nbr_ValorTmp

  # Se regresa el valor
  vctOrigen

}


funcInterCambiarFilasMtrx <- function(mtrx, nbr_FilaOrigen, nbr_FilaDestino, nbr_Cols){
  # Intercambia las filas de una matriz
  #
  # Parámetros
  # ----------
  # mtrx : matriz
  #    La matriz donde se realizará el intercambio de filas
  # nbr_FilaOrigen : número
  #    Índice de la fila origen que se moverá a la fila destino
  # nbr_FilaDestino : número
  #    Índice de la fila destino que se moverá a la fila origen
  #
  # Regresa
  # -------
  # matriz
  #    La matriz con los valores intercambiados
  #

  # Se guarda el valor destino
  vct_FilaTmp <- mtrx[nbr_FilaDestino,1:nbr_Cols]

  # Se pone el valor origen hacia el valor destino
  mtrx[nbr_FilaDestino,1:nbr_Cols] <- mtrx[nbr_FilaOrigen,1:nbr_Cols]

  # Se recupera el valor destino original, y se pone en valor origen
  mtrx[nbr_FilaOrigen,1:nbr_Cols] <- vct_FilaTmp

  # Se regresa el valor
  mtrx

}

funcOrdenarEcuaciones <- function(mtrx_A, vct_B){
  # Ordena las ecuaciones del sistema buscando que no quede ningún cero
  # sobre la diagonal principal (no hay garantía de que no quede algún
  # cero sobre la diagonal). Para saber qué fila tomar, se hace una búsqueda
  # sobre cada columna preguntando por la norma infinito de cada vector-columna.
  # Puesto que el sistema de ecuaciones consta tanto de variables como de
  # resultados, es necesario tambén el re-acomodo del vector de resultados.
  #
  # Parámetros
  # ----------
  # mtrx_A : matriz
  #    La matriz que se va a ordenar
  # vct_B : vector
  #    El vector que se va a ordenar.
  #
  # Regresa
  # -------
  # list
  #    Una lista que contiene la matriz a evaluar y el vector de resultados.
  #

  # Variables que se usan dentro de la funcion
  nbr_Filas <- nrow(mtrx_A)
  nbr_Cols <- ncol(mtrx_A)

  # Se barren todas las columnas (iterador j)
  for (j in 1:nbr_Cols){
    # print('Inicia iteracion')

    # Se saca el vector-columna que se usará esta iteración
    vct_Col = mtrx_A[1:nbr_Cols,j]

    # Mostramos el vector-columna con el que trabajaremos
    # print(vct_Col)

    # Se obtiene la norma infinita del vector-columna
    nbr_Norm = Norm(vct_Col, p = Inf)
    # print(nbr_Norm)

    # Se pregunta si el valor es único en el vector-columna
    vct_OrdDesc <- sort(vct_Col, decreasing = TRUE)
    # print(vct_OrdDesc)

    # Si sí es único:
    if (vct_OrdDesc[1] != vct_OrdDesc[2]){
      #print('Es unico')

      # Se obtiene el índice donde está ese valor
      nbr_Index <- match(nbr_Norm,vct_Col)
      # print(nbr_Index)

      # Si el índice es un NA
      if (is.na(nbr_Index)==TRUE){

        # Multiplicamos el valor de la norma infinito por -1
        nbr_Index <- match(nbr_Norm * -1,vct_Col)

      }

      # Para realizar el intercambio de filas, nuestra
      # fila origen será: nbr_Index, y la fila destino: j
      mtrx_A <- funcInterCambiarFilasMtrx(mtrx_A, nbr_Index, j, nbr_Cols)
      vct_B <- funcInterCambiarFilasVct(vct_B, nbr_Index, j)

      # print(mtrx_A)

    } else { # Si no es único:
      #print('Hay empate')

      vct_Bool1 <- (vct_Col==nbr_Norm)

      mtrx_Tmp <- mtrx_A[vct_Bool1,1:nbr_Cols]
      #print(mtrx_Tmp)

      # Barremos el resto de las columans para el desempate
      for (jj in (j+1):nbr_Cols){

        vct_ColDesempate <- mtrx_Tmp[,jj]
        # print(paste0('vct_ColDesempate: ', vct_ColDesempate))

        # Buscaremos el valor mínimo de la siguiente columna
        vct_OrdAsc <- sort(vct_ColDesempate)
        # print(vct_OrdAsc)

        # Si el  mínimo es único:
        if (vct_OrdAsc[1] != vct_OrdAsc[2]){

          # print('es unico')
          # Generamos el vector que nos ayudará a obtener las filas a desempatar
          vct_Bool2 <- (vct_ColDesempate == vct_OrdAsc[1])
          # print(vct_Bool2)

          vct_FilaDesempate <- mtrx_Tmp[vct_Bool2]

          # Ya que se tiene la fila mínima del empate, se obtiene su indice
          # en la matriz_A
          vct_Bool3 <- rowSums(mtrx_A == vct_FilaDesempate[col(mtrx_A)]) == ncol(mtrx_A)
          nbr_Index <- match(TRUE,vct_Bool3)

          # Para realizar el intercambio de filas, nuestra
          # fila origen será: nbr_Index, y la fila destino: j
          mtrx_A <- funcInterCambiarFilasMtrx(mtrx_A, nbr_Index, j, nbr_Cols)
          vct_B <- funcInterCambiarFilasVct(vct_B, nbr_Index, j)

          #print(mtrx_A)
          break

        }

      }

    }

  }

  # Regresamos en una lista, la matriz y vector ordenados
  list(matriz=mtrx_A,vector=vct_B)

}

In [6]:
####################### 2.- Métodos de Jacobi y Gauss_Seidel #################
# Los métodos de Jacobi y Gauss_Seidel permiten resolver
# sistemas de ecuaciones del tipo Ax=b.
# Esto se logra mediante un proceso iterativo.
# El proceso va a iterar hasta que la diferencia entre el vector de resultados
# de la iteración actual y el vector de resultados de la iteración anterior,
# sea menor a un threshold dado o que se alcance el tope de iteraciones.

# Función que obtiene la aproximación de las iteraciones
funcObtenerVctRslt <- function(nbr_MaxIteraciones, n, mtrx_A, vct_B, vct_X0, nbr_Threshold, str_Metodo){
  # Mediante un proceso de iteraciones, actualiza el vector de aproximaciones
  # vct_X0 para lograr la igualdad: mtrx_A * vct_X0 = vct_B
  # El proceso de iteraciones está sujeto a que se cumpla alguna de las
  # siguientes 2 condiciones:
  #   1: Alcanzar el máximo número de iteraciones (especificado en el parámetro
  #     nbr_MaxIteraciones)
  #   2: Lograr llegar a un diferencia entre iteraciones menor al threshold
  #     que se especifica en el parámetro nbr_Threshold.
  # En cuanto se cumpla alguna de dichas condiciones, termina el proceso de
  # iteraciones.
  # Adicionalmente, hay 2 maneras de calcular el vector de resultados, mediante
  # el método Jacobi o Gauss-Seidel. La manera de especificar qué método se
  # quiere emplear es con el parámetro: str_Metodo que se emplea de la
  # siguiente manera:
  # Si el valor de str_Metodo es 'J', se emplea el método Jacobi
  # Si el valor de str_Metodo es 'GS', se emplea el método Gauss-Seidel
  #
  # Parámetros
  # ----------
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar
  # n : Número
  #    Dimensión de filas o columnas de la matriz
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método ha convergido
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para
  #    actualizar el vector de aproximaciones.
  #
  # Regresa
  # -------
  # vector
  #    El vector de aproximaciones actualizado luego del proceso de iteraciones
  #

  # Inicializamos los vectores de control
  vct_X_Act <- vct_X0
  vct_X_Ant <- vct_X0

  # Los siguientes prints son para debuguear, más adelante se eliminarán
  print(paste0('Iteracion ', 0))
  print(vct_X_Act)

  # Máximo número de iteraciones
  for (it in 1:nbr_MaxIteraciones){

    print(paste0('Iteracion ', it))

    # Iteraciones para obtener cada componente del vector de resultados
    for (i in 1:n){

      # Si se pidió usar el método Jacobi
      if (str_Metodo=='J'){
        vct_X_Act[i]=funcObtenerComponente(i, n, mtrx_A, vct_X_Ant, vct_B)
      }

      # Si se pidió usar el método Gauss-Seidel
      if (str_Metodo=='GS'){
        vct_X_Act[i]=funcObtenerComponente(i, n, mtrx_A, vct_X_Act, vct_B)
      }

    }

    print(vct_X_Act)
    
    # Se calcula la convergencia y se usa la norma infinito
    nbr_Numerador <- Norm(vct_X_Act - vct_X_Ant, p = Inf)
    nbr_Denominador <- Norm(vct_X_Act, p = Inf)

    print(paste0('nbr_Numerador: ', nbr_Numerador))
    print(paste0('nbr_Denominador: ', nbr_Denominador))

    nbr_Diff <-  nbr_Numerador / nbr_Denominador
    print(paste0('nbr_Diff: ',nbr_Diff))

    # Si se empiezan a obtener valores NaN, significa que no hay solución
    if (is.na(nbr_Diff)){
      print('No hay convergencia')
      vct_X_Act <- rep(NA, size(vct_X0)[2])
      break
    }

    # Si se llega a una diferencia menor al threshold indicado, salimos del for
    if (nbr_Diff<nbr_Threshold){
      print('Se alcanza el threshold')
      break
    }

    # El vector resultado (k), lo usamos como vector anterior (k-1) para la sigueinte
    # iteraación
    vct_X_Ant <- vct_X_Act

  }

  if (it==nbr_MaxIteraciones){
    print('Se llega al tope de iteraciones')
  }

  # Devolvemos el último vector calculado
  vct_X_Act

}

funcResolverSE <- function(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo){
  # Resuleve un sistema de ecuaciones lineales mediante el método de Jacobi
  # o de Gauss-Seidel. El sistema de ecuaciones sólo se procesará si pasa
  # todas las validaciones requeridas.
  #
  # Parámetros
  # -------
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método ha convergido
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para
  #    actualizar el vector de aproximaciones.
  #
  # Regresa
  # -------
  # vector
  #    El vector de aproximaciones luego del proceso de iteraciones

  # Se agrega condicón para validar que la matriz no sea singular

  # Se inicializa el vector de resultados
  vct_XRslt <- rep(NA, size(vct_X0)[2])

  if(!is.singular.matrix(mtrx_A)){
      
    # Se lee el método a usar
    if (str_Metodo == 'J' || str_Metodo == 'GS'){

      if (str_Metodo=='J'){
        print('Solucion mediante metodo de Jacobi')
      }
      if (str_Metodo=='GS'){
        print('Solucion mediante metodo de Gauss-Seidel')
      }

      print('Matriz A:')
      print(mtrx_A)

      print('Vector b:')
      print(vct_B)

      # Se aplican las validaciones de manera anidada
      if (funcEsVectorValido(mtrx_A, vct_B)){

        if (funcEsVectorValido(mtrx_A, vct_X0)){

          if (funcEsMatrizCuadrada(mtrx_A) == TRUE){

            # Obtenemos la n de la matriz
            n <- nrow(mtrx_A)

            if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

              # Se manda a llamar la función que obtendrá la aproximación
              vct_XRslt <- funcObtenerVctRslt(nbr_MaxIteraciones, n, mtrx_A, vct_B, vct_X0, nbr_Threshold, str_Metodo)

              # Se imprime el resultado
              print('Resultado final: ')
              print(vct_XRslt)

            } else {
              print('La matriz tiene algun cero en la diagonal, comienza ordenamiento')

              # Se ordena la matriz
              lt_Obj <- funcOrdenarEcuaciones(mtrx_A, vct_B)
              mtrx_A <- lt_Obj$matriz
              vct_B <- lt_Obj$vector

              print('Matriz ordenada:')
              print(mtrx_A)

              print('Vector ordenado:')
              print(vct_B)

              if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

                # Se manda a llamar la función que obtendrá la aproximación
                vct_XRslt <- funcObtenerVctRslt(nbr_MaxIteraciones, n, mtrx_A, vct_B, vct_X0, nbr_Threshold, str_Metodo)

                # Se imprime el resultado
                print('Resultado final: ')
                print(vct_XRslt)

                #En caso de encontrar un problema, se imprime
              } else {
                print('Pese al reordenamiento, aun hay ceros en la diagonal')
                }
            }

          } else {
              print('La matriz no cumple con ser de dimensiones nxn')
            }
            
        } else {
          print('El vector de aproximaciones no es de las dimensiones esperadas')
          }
          
      } else {
        print('El vector de resultados no es de las dimensiones esperadas')
        }

    } else{
      print("El metodo especificado no es valido, se espera 'GS' para Gauss-Seidel o 'J' para Jacobi")
      }
      
  } else{
    print('La matriz no puede ser singular')
    }

  # Se devuelve el vector de resultados
  vct_XRslt

}

In [7]:
########################## 3.- Eliminación por Bloques #########################################
# Este método consiste en eliminar un subconjunto de variables y resolver un sistema más pequeño.
# Si el sistema más pequeño asociado a la submatriz puede resolverse con las funciones anteriores
# entonces este método puede tener más eficiencia que uno que no trabaja por bloques.
  #
  # Parámetros
  # -------
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar dentro del método SEL
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método SEL ha convergido
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para resolver 
  #    los procedimientos que requieran SEL.
  #
  # Regresa
  # -------
  # vector
  #    c(x1,x2) como la solución del sistema

# Algoritmo

EliminacionBloques <- function(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo){
# Variables del algoritmo de eliminación por bloques
col.A <- ncol(mtrx_A)
lim <- col.A/2
A11 <- matrix(mtrx_A[1:lim,1:lim], nrow = lim, ncol=lim)
A12 <- matrix(mtrx_A[1:lim,(lim+1):col.A], nrow = lim, ncol=lim)
A21 <- matrix(mtrx_A[(lim+1):col.A,1:lim], nrow = lim, ncol=lim)
A22 <- matrix(mtrx_A[(lim+1):col.A,(lim+1):col.A], nrow = lim, ncol=lim)
b1<- c(vct_B[1:lim])
b2<- c(vct_B[(lim+1):col.A])
X01<- c(vct_X0[1:lim])
X02<- c(vct_X0[(lim+1):col.A])

#1) Calculamos y = invA11*b1 y Y = invA11*A12 a través de los métodos del bloque anterior donde 
#   los sistemas a evaluar en la forma Ax=b son A11*y=b1 y A11*Y=A12 respectivamente.

###Con el método de Jacobi o GaussSeidel
y <- funcResolverSE(A11, b1, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
print("y:")
print(y)
    
#No podemos resolver A11Y=A12 como tal cual con nuestro método iterativo, pues Y es matriz y el lado derecho (A12) también.
#Es necesario partirla en problemas donde la incógnita y el lado derecho sean vectores.
#Debemos resolver para cada columna de Y, la ecuación A11Yi=A12i , donde Yi es la columan i de Y y A12i es la columa i de A12.
#Así ya podemos aplicar a cada uno de estos SEL nuestro método Jacobi Gauss-Seidel, y al final juntar las soluciones en forma columnar para obtener la matriz Y.    

cols.A12 <- ncol(A12)
rows.A12 <- nrow(A12)

Ycol <- A12[,1]
Ysol <- funcResolverSE(A11, Ycol, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
Y <- Ysol

for (i in 2:cols.A12){
Ycol <- A12[,i]
Ysol <- funcResolverSE(A11, Ycol, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
Y <- matrix(cbind(Y,Ysol), nrow = rows.A12, ncol= i)
}
print("Y")
print(Y)

#2) Calcular el complemento de Schur del bloque A11 en A de la siguiente forma
# Calculamos S
S=A22-(A21%*%Y)
print("S:")
print(S)
    
# Calculamos b_hat
b_hat=b2-(A21%*%y)
print("b_hat")
print(b_hat)
    
#3) Obtenemos x2 resolviendo Sx2=b_hat
### Con el método de Jacobi o GaussSeidel
x2 <- funcResolverSE(S, b_hat, X02, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
print("x2:")
print(x2)

#4) Obtenemos x1 resolviendo A11x1=b1-(A12*x2)
###Con el método de Jacobi o GaussSeidel
x1 <- funcResolverSE(A11, b1-(A12%*%x2) , X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
print("x1:")
print(x1)
    
c(x1,x2)
    
}

funcEliminacionBloques <- function(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo){
  # Aquellos procedimientos que utilicen SEL utilizarán los métodos de Jacobi o GaussSeidel.
  # El sistema de ecuaciones sólo se procesará si pasa
  # todas las validaciones requeridas.
  #
  # Parámetros
  # -------
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar dentro del método SEL (Jacobi o GaussSiedel)
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método ha convergido dentro del método SEL (Jacobi o GaussSiedel)
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para
  #    actualizar el vector de aproximaciones dentro del método SEL (Jacobi o GaussSiedel)
  #
  # Regresa
  # -------
  # vector
  #    x1 como la solución del sistema

  # Condición para validar que la matriz no sea singular

  # Se inicializa el vector de resultados
  #vct_XRslt <- rep(NA, size(vct_X0)[2])

  if(!is.singular.matrix(mtrx_A)){

    if (str_Metodo == 'J' || str_Metodo == 'GS'){

      if (str_Metodo=='J'){
        print('Solucion mediante metodo de Jacobi')
      }
      if (str_Metodo=='GS'){
        print('Solucion mediante metodo de Gauss-Seidel')
      }

      print('Matriz A:')
      print(mtrx_A)

      print('Vector b:')
      print(vct_B)

      # Se aplican las validaciones de manera anidada
      if (funcEsVectorValido(mtrx_A, vct_B)){

        if (funcEsVectorValido(mtrx_A, vct_X0)){

          if (funcEsMatrizCuadrada(mtrx_A) == TRUE){

            # Obtenemos la n de la matriz
            n <- nrow(mtrx_A)

            if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

              # Se manda a llamar la función de eliminación por bloques
              x1_res <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

              # Se imprime el resultado
              print('Resultado final (x1): ')
              print(x1_res)

            } else {
              print('La matriz tiene algun cero en la diagonal, comienza ordenamiento')

              # Se ordena la matriz
              lt_Obj <- funcOrdenarEcuaciones(mtrx_A, vct_B)
              mtrx_A <- lt_Obj$matriz
              vct_B <- lt_Obj$vector

              print('Matriz ordenada:')
              print(mtrx_A)

              print('Vector ordenado:')
              print(vct_B)

              if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

                # Se manda a llamar la función del método de eliminación por bloques
                x1_res <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

                # Se imprime el resultado
                print('Resultado final: ')
                print(x1_res)

                # En caso de encontrar algún problema, se imprime 
              } else {
                print('Pese al reordenamiento, aun hay ceros en la diagonal')
                }
            }

          } else {
              print('La matriz no cumple con ser de dimensiones nxn')
            }
            
        } else {
          print('El vector de aproximaciones no es de las dimensiones esperadas')
          }
          
      } else {
        print('El vector de resultados no es de las dimensiones esperadas')
        }

    } else{
      print("El metodo especificado no es valido, se espera 'GS' para Gauss-Seidel o 'J' para Jacobi")
      }
      
  } else{
    print('La matriz no puede ser singular')
    }

  # Se devuelve x1
  x1_res

}


In [8]:
################### 4.- Declaración de variables y pruebas finales ##########################

# Construcción de los objetos matemáticos

# Ejemplo de Erick (en desorden)
# Matriz A
# mtrx_A <- matrix(c( 0,3,-1,8,
#                    -1,11,-1,3,
#                     2,-1,10,-1,
#                    10,-1,2,0),
#                 byrow = TRUE,
#                 nrow=4,
#                 ncol=4)

# vct_B <- c(15, 25, -11, 6)
# vct_B <- c(6, 25, -11, 15)

# Ejemplo que puso Ana en el chat
# Matriz A
mtrx_A <- matrix(c(2,1,0,0,
                   0,1,2,1,
                   1,2,0,0,
                   1,1,1,0),
                   nrow=4,
                   ncol=4,
                  byrow = TRUE)

# Vector b
vct_B <- c(10, 24, 31, 45)

# Vector de aproximación inicial
vct_X0 <- c(0,0,0,0)

# Máximo número de iteraciones permitido
nbr_MaxIteraciones <- 100

# Threshold que se busca alcanzar
nbr_Threshold <- 10**(-3)

# Método a utilizar
# str_Metodo <- 'J'
str_Metodo <- 'GS'

##################### Pruebas del método de Jacobi o Gauss-Seidel ##########################

# Se manda a llamar la función que contiene las validaciones y ejecución
# del método de Jacobi o Gauss-Seide, según se indique
# vct_Solucion <- funcResolverSE(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

##################### Pruebas del método de eliminación por bloques #######################

# Se manda a llamar la función que contiene las validaciones y ejecución
# del método de Jacobi o Gauss-Seidel, según se indique#x1_Solucion <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
x1_Solucion <- funcEliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

[1] "Solucion mediante metodo de Gauss-Seidel"
[1] "Matriz A:"
     [,1] [,2] [,3] [,4]
[1,]    2    1    0    0
[2,]    0    1    2    1
[3,]    1    2    0    0
[4,]    1    1    1    0
[1] "Vector b:"
[1] 10 24 31 45
[1] "La matriz tiene algun cero en la diagonal, comienza ordenamiento"
[1] "Matriz ordenada:"
     [,1] [,2] [,3] [,4]
[1,]    2    1    0    0
[2,]    1    2    0    0
[3,]    1    1    1    0
[4,]    0    1    2    1
[1] "Vector ordenado:"
[1] 10 31 45 24
[1] "Solucion mediante metodo de Gauss-Seidel"
[1] "Matriz A:"
     [,1] [,2]
[1,]    2    1
[2,]    1    2
[1] "Vector b:"
[1] 10 31
[1] "Iteracion 0"
[1] 0 0
[1] "Iteracion 1"
[1]  5 13
[1] "nbr_Numerador: 13"
[1] "nbr_Denominador: 13"
[1] "nbr_Diff: 1"
[1] "Iteracion 2"
[1] -1.50 16.25
[1] "nbr_Numerador: 6.5"
[1] "nbr_Denominador: 16.25"
[1] "nbr_Diff: 0.4"
[1] "Iteracion 3"
[1] -3.1250 17.0625
[1] "nbr_Numerador: 1.625"
[1] "nbr_Denominador: 17.0625"
[1] "nbr_Diff: 0.0952380952380952"
[1] "Iteracion 4"
[1] -3.53

ERROR: Error in if (!is.singular.matrix(mtrx_A)) {: missing value where TRUE/FALSE needed
